# Discussion

The EIA Form 923 Power Plant Operations Report collects operational information for every electric generating facility in the United States. Among the wealth of information contained in the report is electricity generation by fuel type annually. This information can be helpful to understanding the present condition and recent evolution of the electric power industry across the United States. Knowledge of the existing electicity generation portfolio is essential to understanding the challenges and opportunites presented in transitioning the electric grid to a more sustainable low carbon future in the coming decades. 

The EIA publishes plant level generally data annually in an excel spreadsheet format. While this information is in itself incrediblely useful, I wanted to develop a visualization that could allow for the easy comparison of multiple years of data. Tableau allows users to easily visualize large datasets through dynmaic dashboards and maps. The mapping feature in Tableau allowed me to leverage the plant level geographic corridonate data available from EIA in the Form 860. 

The primary challenge addressed in this notebook is concatenating multiple years of EIA 923 plant level data, linking it to the corresponding EIA 860 data, cleaning and streamlining the data, and arranging it into a format where it can easily be read into a Tableau workbook. This process involved summarizing the various EIA fuel and prime mover codes, calculating an estimate of corresponding CO2 emissions for fossil fueled plant's, and some plant specific adjustments to improve the visualization. 


https://public.tableau.com/app/profile/michael.leff/viz/USGenerationbyFuel/StateDashboard_1

# Import EIA 923 Data

In [5]:
import pandas as pd
import numpy as np
import os

This process involves downloading each respective annual 923 dataset from the EIA's 923 page and placing the spreadsheet's in each respective folder
https://www.eia.gov/electricity/data/eia923/

In [2]:
#os.chdir(r'DEFINE USER PATH TO EIA 923 FILES')

Read in annual EIA 923 forms from 2020 through 2012. Use a for loop to read in forms for each year. Initially read in the tables as a list and then concatenate the lists to create an appended pandas dataframe. 2013 & 2011 forms had slighly difference names so they are added in a 2nd for loop. 

In [5]:
years1 = ['2020','2019','2018','2017','2016','2015','2014','2012']

appended_data = []

for year in years1:
    data = pd.read_excel('EIA923_Schedules_2_3_4_5_M_12_'+year+'_Final_Revision.xlsx',header=5, usecols='A,D:S,CN:CS',
                        sheet_name='Page 1 Generation and Fuel Data')
    appended_data.append(data)
    df_in = pd.concat(appended_data)

In [6]:
years2 = ['2013','2011']

appended_data2 = []

for year in years2:
    data2 = pd.read_excel('EIA923_Schedules_2_3_4_5_'+year+'_Final_Revision.xlsx',header=5, usecols='A,D:S,CN:CS')
    appended_data2.append(data2)
    df_in2 = pd.concat(appended_data2)

Read in 2010 data

In [7]:
years3 = ['2010']

appended_data3 = []

for year in years3:
    data3 = pd.read_excel('EIA923 SCHEDULES 2_3_4_5 Final '+year+'.xls',header=7, usecols='A,D:S,CN:CS', sheet_name='Page 1 Generation and Fuel Data')
    appended_data3.append(data3)
    df_in3 = pd.concat(appended_data3)

Rename tables in the 2010 dataset to match the other years

In [8]:
df_in3 = df_in3.rename(columns={'Plant ID':'Plant Id','Operator ID':'Operator ID',
                       'TOTAL FUEL CONSUMPTION QUANTITY':'Total Fuel Consumption Quantity',
                       'ELECTRIC FUEL CONSUMPTION QUANTITY':'Electric Fuel Consumption Quantity',
                       'TOTAL FUEL CONSUMPTION MMBTUS':'Total Fuel Consumption MMBtu',
                       'ELEC FUEL CONSUMPTION MMBTUS':'Elec Fuel Consumption MMBtu',
                       'NET GENERATION (megawatthours)':'Net Generation (Megawatthours)',
                       'Year':'YEAR'})

The following steps normalize headers across all annual datasets. The first step defines a function which renames each header by removing spaces from both headers. The remaining steps utilize this function and rename each column to match each other.  

In [9]:
def header_update(old_header):
    empty_list = []
    for element in old_header:
        new_element = element.replace('\n',' ')
        empty_list.append(new_element)
    return empty_list

In [10]:
header_923_1 = df_in.columns.tolist()
header_923_2 = df_in2.columns.tolist()
header_923_3 = df_in3.columns.tolist()

In [11]:
new_923_header = header_update(header_923_1)
new_923_header2 = header_update(header_923_2)
new_923_header3 = header_update(header_923_3)

In [12]:
df_in.columns = new_923_header
df_in2.columns = new_923_header2
df_in3.columns = new_923_header3

In [13]:
df_in2 = df_in2.rename(columns={'State':'Plant State'})
df_in3 = df_in3.rename(columns={'State':'Plant State'})

In [14]:
eia_923 = pd.concat([df_in, df_in2, df_in3], axis=0)

Filter out rows with no year. 

In [15]:
eia_923 = eia_923[eia_923['YEAR']!='.']

In [16]:
df_in3

,Plant Id,Plant Name,Operator Name,Operator ID,Plant State,Census Region,NERC Region,Reserved,NAICS Code,EIA Sector Number,...,AER Fuel Type Code,Reserved .1,Reserved .2,Physical Unit Label,Total Fuel Consumption Quantity,Electric Fuel Consumption Quantity,Total Fuel Consumption MMBtu,Elec Fuel Consumption MMBtu,Net Generation (Megawatthours),YEAR
0,2,Bankhead Dam,Alabama Power Co,195,AL,ESC,SERC,NaN,22,1,...,HYC,NaN,NaN,NaN,0,0,985172,985172,100981.000,2010
1,3,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,22,1,...,NG,NaN,NaN,mcf,399953,399953,409113,409113,2605623.000,2010
2,3,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,22,1,...,DFO,NaN,NaN,barrels,0,0,0,0,0.000,2010
3,3,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,22,1,...,NG,NaN,NaN,mcf,49297741,49297741,50455319,50455319,4748932.000,2010
4,3,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,22,1,...,COL,NaN,NaN,short tons,3476163,3476163,78476870,78476870,7749280.005,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10103,99999,State-Fuel Level Increment,State-Fuel Level Increment,99999,OH,ENC,NaN,NaN,99999,7,...,WWW,NaN,NaN,short tons,0,0,0,0,0.000,2010
10104,99999,State-Fuel Level Increment,State-Fuel Level Increment,99999,OR,PACC,NaN,NaN,99999,5,...,NG,NaN,NaN,mcf,0,0,0,0,0.000,2010
10105,99999,State-Fuel Level Increment,State-Fuel Level Increment,99999,OR,PACC,NaN,NaN,99999,5,...,NG,NaN,NaN,mcf,0,0,0,0,0.000,2010
10106,99999,State-Fuel Level Increment,State-Fuel Level Increment,99999,TN,ESC,NaN,NaN,99999,5,...,NG,NaN,NaN,mcf,499180,231652,510450,236186,12063.000,2010


# Read in Fuel & Unit Type Summary

This file reads in the EIA fuel type and summarizes it. For example there are 5 types of EIA fuel types representing coal. For display purposes it's easiest to classify fuel type at their summary value. The fuel type summary file is shown in the notebook. A similar summarization is made using the EIA Prime Mover code.  

In [4]:
#os.chdir(r'DEFINE USER PATH FUEL TYPE SUMMARY FILE')

In [18]:
fuel_type_summary = pd.read_excel('fuel_type_summary.xlsx',sheet_name='Fuel')
fuel_type_summary

,Reported Fuel Type Code,Fuel Type Summary
0,WAT,Hydro
1,DFO,Oil
2,BIT,Coal
3,NG,Natural Gas
4,RFO,Oil
5,KER,Oil
6,NUC,Nuclear
7,OTH,Other
8,WO,Oil
9,WDS,Biomass


In [19]:
unit_type_summary = pd.read_excel('fuel_type_summary.xlsx',sheet_name='Unit')
unit_type_summary

,Reported Prime Mover,Unit Type Summary
0,IC,Internal Combustion
1,ST,Steam Turbine
2,CA,Combined Cycle
3,CT,Combined Cycle
4,GT,Gas Turbine
5,CS,Combined Cycle
6,FC,Fuel Cells
7,BA,Energy Storage
8,CE,Energy Storage
9,CP,Solar


# Calculate CO2 emissions by fuel type and unit type

CO2 emissions rate's for fossil fueled electric generating plants were calculated by utilizing the respective heat rate and the carbon content from each respective fuel. Heat rate data by prime mover and energy source is found in the table 8.2 of EIA's Electric Power Annual. The carbon content of each fuel is found in Table A 3 of EIA's Electric Power Annual. 

https://www.eia.gov/electricity/annual/html/epa_08_02.html
https://www.eia.gov/electricity/annual/html/epa_a_03.html

In order to be able to convert electricity generation by fuel type and unit type to CO2 emissions, a CO2 emissions rate in metric tons/MWh was calculated by the unit type emissions rate and carbon content of each respective fuel. This conversion from (BTU/kWh) to (kg/mmBTU) by each EIA fuel type and EIA prime mover can be seen below. 

In [49]:
co2_rate_in = pd.read_excel('fuel_type_summary.xlsx',sheet_name='Emissions Rate Summary')
co2_rate_in

,Reported Prime Mover,Reported Fuel Type Code,Fuel Type Summary,Plant Type Summary,Unit Type Emissions Rate (BTU/kWh),Carbon Content (kg/mmBTU),Fuel Unit Lookup,metric tons/ (MWh)
0,IC,ANT,Coal,Coal_IC,10015,93.30,ANT_IC,0.934399
1,ST,ANT,Coal,Coal_ST,10015,93.30,ANT_ST,0.934399
2,CA,BIT,Coal,Coal_CA,10015,93.30,BIT_CA,0.934399
3,CT,BIT,Coal,Coal_CT,10015,93.30,BIT_CT,0.934399
4,GT,BIT,Coal,Coal_GT,10015,93.30,BIT_GT,0.934399
5,ST,BIT,Coal,Coal_ST,10015,93.30,BIT_ST,0.934399
6,CA,DFO,Oil,Oil_CA,9663,73.16,DFO_CA,0.706945
7,CT,DFO,Oil,Oil_CT,9663,73.16,DFO_CT,0.706945
8,GT,DFO,Oil,Oil_GT,13352,73.16,DFO_GT,0.976832
9,IC,DFO,Oil,Oil_IC,10326,73.16,DFO_IC,0.755450


In [52]:
co2_rate = co2_rate_in[['Fuel Type Summary','Plant Type Summary','Unit Type Emissions Rate (BTU/kWh)','Carbon Content (kg/mmBTU)',
            'Fuel Unit Lookup','metric tons/ (MWh)']]

# EIA 860

Read in EIA 860 Files from 2012 and 2020. Filter out only the Plant Code and Latitude & Longtitude. This will make joining the latitute and longitude with the 923 dataframe easier in the next step. They are then concatenated in addition to plant county information. The 2012 EIA 860 and 2020 EIA 860 should capture the locational information for all units operating over the last 10 years.

https://www.eia.gov/electricity/data/eia860/

In [53]:
plant_860_20 = pd.read_excel('2___Plant_Y2020.xlsx', header=1, usecols='A:K')
plant_860_20.drop('Utility Name', axis=1, inplace=True)
plant_860_20.head()

,Utility ID,Plant Code,Plant Name,Street Address,City,State,Zip,County,Latitude,Longitude
0,63560,1,Sand Point,100 Power Plant Way,Sand Point,AK,99661,Aleutians East,55.339722,-160.497222
1,195,2,Bankhead Dam,19001 Lock 17 Road,Northport,AL,35476,Tuscaloosa,33.458665,-87.356823
2,195,3,Barry,North Highway 43,Bucks,AL,36512,Mobile,31.0069,-88.0103
3,195,4,Walter Bouldin Dam,750 Bouldin Dam Road,Wetumpka,AL,36092,Elmore,32.583889,-86.283056
4,195,7,Gadsden,1000 Goodyear Ave,Gadsden,AL,35903,Etowah,34.0128,-85.9708


filter out only fields used in join

In [54]:
plant_860_20a = plant_860_20[['Plant Code','Latitude','Longitude','County']]
plant_860_20b = plant_860_20[['Plant Code']]

Read in 2012 EIA 860 data

In [55]:
plant_860_12 = pd.read_excel('PlantY2012.xlsx',header=1, usecols='A:H,AA:AB')

In [56]:
plant_860_12a = plant_860_12[['Plant Code','Latitude','Longitude','County']]
plant_860_12b = plant_860_12[['Plant Code']]

Concatenate the Plant Code series from the 2020 860 form and the 2012 860 form

In [57]:
plant_codes = pd.concat([plant_860_20b, plant_860_12b], axis=0).drop_duplicates()


Merge the complete list of plant codes with the df containing lat/long's and counties in both the 2020 and 2012 860 tables. Left merge the 2012 table first and then left merge 2012 list with the 2020 860 data to fill in the NA's 

In [58]:
plant_860a = pd.merge(plant_codes, plant_860_12a, how='left', on='Plant Code')
plant_860b = pd.merge(plant_codes, plant_860_20a, how='left', on='Plant Code')

In [59]:
plant_860b[plant_860b['Plant Code']==56047]

,Plant Code,Latitude,Longitude,County
4683,56047,41.481243,-73.123108,New Haven


# Merge 923, 860, Fuel Type Summary, Unit Type Summary, & CO2 emissions rate to one dataframe

In [60]:
df_923a = pd.merge(eia_923, fuel_type_summary, how='left', left_on ='Reported Fuel Type Code',
                                                     right_on='Reported Fuel Type Code')

Left join in Unit Type Summary parameters into main table

In [61]:
df_923b = pd.merge(df_923a, unit_type_summary, how='left', left_on ='Reported Prime Mover', right_on='Reported Prime Mover')

Left join in 2020 EIA 860 data to main table on Plant Id in the 923 & Plant Code in the 860

In [62]:
df_923c = pd.merge(df_923b, plant_860b, how='left', left_on =['Plant Id'], right_on=['Plant Code'])

Create Fuel Type lookup function with concatenates prime move and reported fuel type code. This creates a referernce for emissions rate by fuel type and unit type. For example a natural gas combined cycle plant will have a lower emissiosn rate than a natural gas steam turbine plant

In [63]:
df_923c['Fuel Unit Lookup'] = df_923c['Reported Fuel Type Code'] + "_" + df_923c['Reported Prime Mover']

Merge emissions rates by Fuel Unit Lookup parameter to the main table

In [78]:
df_923d = pd.merge(df_923c, co2_rate, how='left', on=['Fuel Unit Lookup','Fuel Type Summary'])

Calculate an estimate of annual CO2 emissions by plant by multiplying emissions rate by net generation

In [79]:
df_923d['CO2 emissions - MT (Est)'] = df_923d['metric tons/ (MWh)'] * df_923d['Net Generation (Megawatthours)']

Parse out lower 48 state data

In [80]:
df_923e = df_923d[(df_923d['Plant State']!='AK')&(df_923d['Plant State']!='HI')]

# Adjustment to Indian Point Nuclear Power Units 2 & 3

Combine Indian Point Units 2 & 3 by temporarily deleting them and then combining them again. This is to ensure that they have the appropriate scale effect on the map in Tableau

Filter out indian point unit's 2 and 3 from the final dataframe. Will add back the combined Indian Point generation in subsequent steps

In [84]:
df_923f = df_923e[(df_923e['Plant Name']!='Indian Point 3')&(df_923e['Plant Name']!='Indian Point 2')]

Parse out Indian Point Units 2 & 3

In [85]:
ip1 = df_923e[(df_923e['Plant Name']=='Indian Point 3')|(df_923e['Plant Name']=='Indian Point 2')]
ip1[['Plant Name','Plant Id','YEAR','Net Generation (Megawatthours)']].head()

,Plant Name,Plant Id,YEAR,Net Generation (Megawatthours)
2230,Indian Point 2,2497,2020,2759083.0
4706,Indian Point 3,8907,2020,9108821.0
18147,Indian Point 2,2497,2019,8351945.0
20647,Indian Point 3,8907,2019,8342898.0
33423,Indian Point 2,2497,2018,8018479.0


Combine Indian Point Units 2 & 3 by setting the fields which difrerentiate the two as equal. This step is taken to 

In [86]:
ip1['Plant Id'] = 2497
ip1['Plant Name'] = 'Indian Point'
ip1['Operator Name'] = 'Entergy Nuclear Indian Point'
ip1['Operator Id'] = 6028
ip1['Latitude'] = 41.270600
ip1['Longitude'] = -73.9527
ip1['Plant Code'] = 2497
ip1['County'] = 'Westchester'

C:\Users\mikel\AppData\Local\Temp/ipykernel_2612/852008358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ip1['Plant Id'] = 2497
C:\Users\mikel\AppData\Local\Temp/ipykernel_2612/852008358.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ip1['Plant Name'] = 'Indian Point'
C:\Users\mikel\AppData\Local\Temp/ipykernel_2612/852008358.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [88]:
ip1_cols = ip1.columns.tolist()

In [89]:
ip_col_values = ip1_cols[18:22]

In [90]:
ip_col_index = ['Plant Id', 'Plant Name', 'Operator Name', 'Operator Id', 'Plant State', 'Census Region', 'NERC Region', 'Reported Prime Mover', 'Reported Fuel Type Code', 'AER Fuel Type Code','Fuel Type Summary', 
                'Unit Type Summary','Plant Code','County','Latitude','Longitude','YEAR']

In [99]:
ip2 = ip1.pivot_table(index=ip_col_index, values=ip_col_values, aggfunc=np.sum).reset_index()
ip2[['Plant Name','YEAR','Net Generation (Megawatthours)']]

,Plant Name,YEAR,Net Generation (Megawatthours)
0,Indian Point,2011,17016862.0
1,Indian Point,2012,16937052.0
2,Indian Point,2013,17076155.0
3,Indian Point,2014,17308255.0
4,Indian Point,2015,16421167.0
5,Indian Point,2016,15126169.0
6,Indian Point,2017,15302630.0
7,Indian Point,2018,16318960.0
8,Indian Point,2019,16694843.0
9,Indian Point,2020,11867904.0


Concatenate Indian Point revisions

In [93]:
df_923g = pd.concat([df_923f, ip2], axis=0)

In [ ]:
os.chdir(r'DEFINE LOCATION TO USER OUTPUT')

Simplify dataframe to only use columns needed in the Tableau visualization

In [95]:
df_923h = df_923g[['Plant Id','Plant Name','Plant State','Census Region','NERC Region','YEAR','County',
         'Fuel Type Summary','Unit Type Summary','Net Generation (Megawatthours)',
         'Latitude','Longitude','CO2 emissions - MT (Est)']]

Download dataframe to be used in tableau visualization

In [97]:
df_923h.to_csv('eia_923.csv')

# 